Connect google drive

In [17]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [18]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [25]:
# no need unzip anymore
!unzip '/content/drive/MyDrive/SDGP/Individual Folders/Thilina/ML Model/Datasets/asl_alphabet_test.zip' -d '/content/drive/MyDrive/SDGP/Individual Folders/Thilina/ML Model/Datasets/'

Archive:  /content/drive/MyDrive/SDGP/Individual Folders/Thilina/ML Model/Datasets/asl_alphabet_test.zip
replace /content/drive/MyDrive/SDGP/Individual Folders/Thilina/ML Model/Datasets/asl_alphabet_test/space_test.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [20]:
# no need unzip anymore
!unzip '/content/drive/MyDrive/SDGP/Individual Folders/Thilina/ML Model/Datasets/asl_alphabet_train.zip' -d '/content/drive/MyDrive/SDGP/Individual Folders/Thilina/ML Model/Datasets/'

Archive:  /content/drive/MyDrive/SDGP/Individual Folders/Thilina/ML Model/Datasets/asl_alphabet_train.zip
replace /content/drive/MyDrive/SDGP/Individual Folders/Thilina/ML Model/asl_alphabet_train/space/space130.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# **Import Libraries**

In [21]:
# To check if GPU is active
from tensorflow.python.client import device_lib

# Load Data
import os
import cv2
import numpy as np

# Data Visualisation
import matplotlib.pyplot as plt

# Model Training
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split

In [22]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5165519605182494511
xla_global_id: -1
]


# **1. Load Data**

In [23]:
train_dir = '/content/drive/MyDrive/SDGP/Individual Folders/Thilina/ML Model/asl_alphabet_train'
test_dir = '/content/drive/MyDrive/SDGP/Individual Folders/Thilina/ML Model/asl_alphabet_test'

In [ ]:
def get_data(data_dir):
  images = []
  labels = []
  
  dir_list = os.listdir(data_dir)
  for i in range(len(dir_list)):
    print("Object images of", dir_list[i], "...")
    for image in os.listdir(data_dir + "/" + dir_list[i]):
      img = cv2.imread(data_dir + '/' + dir_list[i] + '/' + image)
      img = cv2.resize(img, (32, 32))
      images.append(img)
      labels.append(i)
  return images, labels

X, y = get_data(train_dir)


In [ ]:
print(len(X), len(y))

# **2. Data Visualisation**

In [ ]:
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

In [ ]:
def plot_sample_images():
    figure = plt.figure()
    plt.figure(figsize=(16,5))

    for i in range (0,29):
        plt.subplot(3,10,i+1)
        plt.xticks([])
        plt.yticks([])
        path = train_dir + "/{0}/{0}1.jpg".format(classes[i])
        img = plt.imread(path)
        plt.imshow(img)
        plt.xlabel(classes[i])
        
plot_sample_images()

# **3. Data Processing**

In [ ]:
def preprocess_data(X,y):
  np_X = np.array(X)
  normalised_X = np_X.astype('float32')/255.0
  
  label_encoded_y = utils.to_categorical(y)

  x_train, x_test, y_train, y_test = train_test_split(normalised_X, label_encoded_y, test_size = 0.1)

  return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = preprocess_data(X,y)


In [ ]:
print ("Training data:" , x_train.shape)
print("Test data:" , x_test.shape)

# **4. Define CNN Model**

In [ ]:
classes = 29
batch = 32
epochs = 15
learning_rate = 0.001

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dense(classes, activation='softmax'))

In [ ]:
# Adam is used as the optimizer here with 0.001 as its learning rate.
# Since this is a multiclass classification, categorical crossentropy is chosen as the loss function. The model is trained on the metric of accuracy.

adam = Adam(lr=learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

# **5. Train Model**

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch, 
                    epochs=epochs, validation_split = 0.2, shuffle=True, verbose=1)

# **6. Evaluate Model**

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Testing accuracy:', test_acc)
print('Test loss:', test_loss)

In [ ]:
# Now let us plot accuracy and loss over the training epochs to better understand the training of our model.
def plot_results(model):

  plt.figure(figsize=(12, 12))
  plt.subplot(3, 2, 1)
  plt.plot(history.history['accuracy'], label = 'train_accuracy')
  plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.legend()
  plt.subplot(3, 2, 2)
  plt.plot(history.history['loss'], label = 'train_loss')
  plt.plot(history.history['val_loss'], label = 'val_loss')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.legend()
  plt.show()
  